In [1]:
data1 = [(7369, "SMITH", "CLERK", 7902, "17-12-1980", 800, None, 20),
        (7499, "ALLEN", "SALESMAN",7698,"20-02-1981",1600,300,  30),
        (7521, "WARD", "SALESMAN", 7698,"22-02-1981",1250,500,  30),
        (7566, "JONES", "MANAGER", 7839, "02-04-1981",2975,None,20),
        (7654, "MARTIN", "SALESMAN", 7698,"28-09-1981",1250,1400,30),
        (7698, "BLAKE", "MANAGER", 7839, "01-05-1981", 2850, None,30),
        (7782, "CLARK", "MANAGER", 7839, "09-06-1981", 2450, None, 10),
        (7788, "SCOTT", "ANALYST", 7566, "09-12-1982", 3000, None, 20),
        (7839, "KING", "PRESIDENT",None, "17-11-1981", 5000, None, 10),
        (7844, "Turner", "SALESMAN",7698, "08-09-1981", 1500, 0,  30),
        (7876, "ADAMS", "CLERK",   7788,  "12-01-1983", 1100, None, 20),
        (7900, "JAMES",  "CLERK",  7698,  "03-12-1981", 950, None, 30),
        (7902, "FORD",  "ANALYST", 7566,  "03-12-1981", 3000, None, 20),
        (7934, "MILLER", "CLERK",  7782,  "23-01-1982", 1300, None, 10)
       ]
schema1 = ["empno","ename","job","mgr","hiredate","sal","comm","deptno"]
        

In [2]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import *
spark = SparkSession.builder.getOrCreate()
emp_df = spark.createDataFrame(data1,schema1)

24/12/26 16:24:44 WARN Utils: Your hostname, user-HP-Pavilion-x360-Convertible-14-dh0xxx resolves to a loopback address: 127.0.1.1; using 192.168.122.1 instead (on interface virbr0)
24/12/26 16:24:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/26 16:24:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/26 16:24:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/26 16:24:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
emp_df.createOrReplaceTempView('emp')

In [4]:
data2 = [(10, "ACCOUNTING", "NEW YORK"),
         (20, "RESEARCH", "DALLAS"),
         (30, "SALES",    "CHICAGO"),
         (40, "OPERATIONS", "BOSTON") ]
schema2 = ["deptno", "dname", "loc"]

In [5]:
dept_df = spark.createDataFrame(data = data2, schema = schema2) 

In [6]:
dept_df.createOrReplaceTempView('dept')

In [ ]:
spark.sql('''select sal from (
             select row_number() over(order by sal) as rn, sal 
             from emp) x 
             where rn between 1 and 5''').show()

In [7]:
from pyspark.sql.window import Window 
windowspec = Window.orderBy(desc("sal")) 
emp_df.withColumn("row_number",row_number().over(windowspec)).where(col("row_number")==1).show()

24/12/26 16:25:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:25:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:25:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+-----+---------+----+----------+----+----+------+----------+
|empno|ename|      job| mgr|  hiredate| sal|comm|deptno|row_number|
+-----+-----+---------+----+----------+----+----+------+----------+
| 7839| KING|PRESIDENT|NULL|17-11-1981|5000|NULL|    10|         1|
+-----+-----+---------+----+----------+----+----+------+----------+



# Skipping n Rows from a Table

In [ ]:
spark.sql('''select ename 
               from (
               select row_number() over (order by ename) rn, ename 
               from emp) x
               where mod(rn, 2) = 1 ''').show()

In [8]:
from pyspark.sql.window import Window
windowspec = Window.orderBy("sal") 

emp_df.withColumn('rn',row_number().over(windowspec)).where(col('rn') % 2 ==1).select('rn','ename').show()

24/12/26 16:25:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:25:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:25:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+------+
| rn| ename|
+---+------+
|  1| SMITH|
|  3| ADAMS|
|  5|MARTIN|
|  7|Turner|
|  9| CLARK|
| 11| JONES|
| 13|  FORD|
+---+------+



24/12/26 16:25:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:25:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


# Incorporating OR Logic When Using Outer Joins

In [9]:
dept_df.show()

+------+----------+--------+
|deptno|     dname|     loc|
+------+----------+--------+
|    10|ACCOUNTING|NEW YORK|
|    20|  RESEARCH|  DALLAS|
|    30|     SALES| CHICAGO|
|    40|OPERATIONS|  BOSTON|
+------+----------+--------+



In [10]:
dept_df.join(emp_df, (dept_df.deptno==emp_df.deptno) & ((emp_df.deptno==10)|(emp_df.deptno==20)) , "left_outer").show() #where((emp_df.deptno == 10)|(emp_df.deptno == 20)).show()

+------+----------+--------+-----+------+---------+----+----------+----+----+------+
|deptno|     dname|     loc|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+------+----------+--------+-----+------+---------+----+----------+----+----+------+
|    10|ACCOUNTING|NEW YORK| 7934|MILLER|    CLERK|7782|23-01-1982|1300|NULL|    10|
|    10|ACCOUNTING|NEW YORK| 7839|  KING|PRESIDENT|NULL|17-11-1981|5000|NULL|    10|
|    10|ACCOUNTING|NEW YORK| 7782| CLARK|  MANAGER|7839|09-06-1981|2450|NULL|    10|
|    20|  RESEARCH|  DALLAS| 7902|  FORD|  ANALYST|7566|03-12-1981|3000|NULL|    20|
|    20|  RESEARCH|  DALLAS| 7876| ADAMS|    CLERK|7788|12-01-1983|1100|NULL|    20|
|    20|  RESEARCH|  DALLAS| 7788| SCOTT|  ANALYST|7566|09-12-1982|3000|NULL|    20|
|    20|  RESEARCH|  DALLAS| 7566| JONES|  MANAGER|7839|02-04-1981|2975|NULL|    20|
|    20|  RESEARCH|  DALLAS| 7369| SMITH|    CLERK|7902|17-12-1980| 800|NULL|    20|
|    30|     SALES| CHICAGO| NULL|  NULL|     NULL|NULL|      NUL

In [ ]:
spark.sql("select * from dept").show()

# Selecting the Top n Records

In [12]:
from pyspark.sql.window import Window
windowspec=Window.orderBy(desc("sal"))
df = emp_df.withColumn('dr',dense_rank().over(windowspec))
df.where(df.dr<=5).show()

24/12/26 16:28:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:28:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:28:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+-----+---------+----+----------+----+----+------+---+
|empno|ename|      job| mgr|  hiredate| sal|comm|deptno| dr|
+-----+-----+---------+----+----------+----+----+------+---+
| 7839| KING|PRESIDENT|NULL|17-11-1981|5000|NULL|    10|  1|
| 7788|SCOTT|  ANALYST|7566|09-12-1982|3000|NULL|    20|  2|
| 7902| FORD|  ANALYST|7566|03-12-1981|3000|NULL|    20|  2|
| 7566|JONES|  MANAGER|7839|02-04-1981|2975|NULL|    20|  3|
| 7698|BLAKE|  MANAGER|7839|01-05-1981|2850|NULL|    30|  4|
| 7782|CLARK|  MANAGER|7839|09-06-1981|2450|NULL|    10|  5|
+-----+-----+---------+----+----------+----+----+------+---+



24/12/26 16:28:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:28:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


# Finding Records with the Highest and Lowest Values

In [13]:
min=emp_df.agg({"sal": "min"}).collect()[0][0]
max=emp_df.agg({"sal":"max"}).collect()[0][0]
lst = [min, max]
print(f'lst = {lst}')
emp_df.where(col("sal").isin(lst)).show()
emp_df.where(col("sal").isin(lst)).select("ename","sal").show()

lst = [800, 5000]
+-----+-----+---------+----+----------+----+----+------+
|empno|ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+-----+---------+----+----------+----+----+------+
| 7369|SMITH|    CLERK|7902|17-12-1980| 800|NULL|    20|
| 7839| KING|PRESIDENT|NULL|17-11-1981|5000|NULL|    10|
+-----+-----+---------+----+----------+----+----+------+

+-----+----+
|ename| sal|
+-----+----+
|SMITH| 800|
| KING|5000|
+-----+----+



# Investigating Future Rows

In [14]:
emp_df=emp_df.withColumn('hiredate', to_date(col("hiredate"),"dd-mm-yyyy"))

In [15]:
from pyspark.sql.window import Window
#windowspec=Window.partitionBy("deptno").orderBy("sal") 
windowspec=Window.orderBy("hiredate") 
#emp_df1.withColumn("next_sal",lead("sal",1).over(windowspec)).show()
emp_df.withColumn("next_sal",lead("sal",1).over(windowspec)).where(col("sal") < col("next_sal")).show()

24/12/26 16:31:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:31:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:31:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+------+--------+----+----------+----+----+------+--------+
|empno| ename|     job| mgr|  hiredate| sal|comm|deptno|next_sal|
+-----+------+--------+----+----------+----+----+------+--------+
| 7369| SMITH|   CLERK|7902|1980-01-17| 800|NULL|    20|    2850|
| 7698| BLAKE| MANAGER|7839|1981-01-01|2850|NULL|    30|    2975|
| 7900| JAMES|   CLERK|7698|1981-01-03| 950|NULL|    30|    3000|
| 7844|Turner|SALESMAN|7698|1981-01-08|1500|   0|    30|    2450|
| 7782| CLARK| MANAGER|7839|1981-01-09|2450|NULL|    10|    5000|
| 7654|MARTIN|SALESMAN|7698|1981-01-28|1250|1400|    30|    3000|
+-----+------+--------+----+----------+----+----+------+--------+



24/12/26 16:31:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:31:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


# Shifting Row Values

In [16]:
from pyspark.sql.window import Window
windspec=Window.orderBy("sal")

emp_df = emp_df.withColumn("forward",lead("sal").over(windspec)) #.show() 
#emp_df.withColumn("rewind", lag("sal").over(windspec)).show() 

min=emp_df.agg({"sal": "min"}).collect()[0][0]
emp_df = emp_df.select('*',lit(min).alias("min")) #.show()
emp_df.show()
emp_df = emp_df.withColumn("forward",coalesce(emp_df.forward,emp_df.min)) #.show()
emp_df.drop("min").show()

24/12/26 16:37:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:37:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:37:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:37:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:37:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:37:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 1

+-----+------+---------+----+----------+----+----+------+-------+---+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|forward|min|
+-----+------+---------+----+----------+----+----+------+-------+---+
| 7369| SMITH|    CLERK|7902|1980-01-17| 800|NULL|    20|    950|800|
| 7900| JAMES|    CLERK|7698|1981-01-03| 950|NULL|    30|   1100|800|
| 7876| ADAMS|    CLERK|7788|1983-01-12|1100|NULL|    20|   1250|800|
| 7521|  WARD| SALESMAN|7698|1981-01-22|1250| 500|    30|   1250|800|
| 7654|MARTIN| SALESMAN|7698|1981-01-28|1250|1400|    30|   1300|800|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|   1500|800|
| 7844|Turner| SALESMAN|7698|1981-01-08|1500|   0|    30|   1600|800|
| 7499| ALLEN| SALESMAN|7698|1981-01-20|1600| 300|    30|   2450|800|
| 7782| CLARK|  MANAGER|7839|1981-01-09|2450|NULL|    10|   2850|800|
| 7698| BLAKE|  MANAGER|7839|1981-01-01|2850|NULL|    30|   2975|800|
| 7566| JONES|  MANAGER|7839|1981-01-02|2975|NULL|    20|   3000|800|
| 7788| SCOTT|  ANAL

24/12/26 16:37:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/26 16:37:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


# Ranking Results

In [ ]:
from pyspark.sql.window import Window
windspec=Window.orderBy("sal")
emp_df1.withColumn("drnk", dense_rank().over(windspec)).select("drnk","sal").show()  

# Finding Knight Values